In [1]:
import pandas as pd
import requests
import re # for regex in html
from bs4 import BeautifulSoup

%load_ext line_profiler

pd.options.display.max_columns=500
pd.options.display.max_rows=1500
pd.options.display.max_colwidth=1000

data_ = pd.read_json('https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2015.json')

In [17]:
data = data_[data_.armed=="unarmed"].copy()
data = data.reset_index()

sources = pd.DataFrame(data["sources"].apply(lambda x:[y["url"] for y in x]).tolist())

for col in sources.columns:
    data["source_"+str(col)]=sources[col]
    
def scraper(URL):
    if URL==None:
        pass
        return
    else:
        r = requests.get(URL)
        soup = BeautifulSoup(r.content)
    
        return soup.prettify()
    
df2015_with_info = data.copy() # for reference, df2015_with_info.shape is (94, 30)

def for_profiler(): # use just code inside the function when not using the profiler
    for source_col in ['source_0','source_1','source_2','source_3','source_4']:
        # this is just on 6 first entries, as a test.
        df2015_with_info.loc[:5,source_col] = df2015_with_info.loc[:5,source_col].apply(scraper)
    return

%lprun -f scraper for_profiler()